In [60]:
import pandas as pd
import numpy as np
import os




In [61]:


# Use the absolute path to the file
#file_path = r'C:\Users\GICHEHA\Documents\Flatiron\dsc-data-science-env-config\phase_5.0\Data\reduced_reddit.csv'




In [62]:
#data = pd.read_csv(file_path)  # 
data = pd.read_csv('../Data/reduced_reddit.csv', index_col=False)
print(data.head())  # Display the first few rows of the dataframe


                                               title  \
0                        Day 18 of doing 50 push-ups   
1  There isn’t a better feeling than finishing yo...   
2  LMAOOO I can only get this guy to talk to me i...   
3                       This isn't going to work out   
4  Am I the only guy who found it hurtful as a ch...   

                                                body  upvotes   created_utc  \
0                                                NaN      4.0  1.619357e+09   
1  Then your teacher hits you with that “ Good jo...      7.0  1.554103e+09   
2                              Yeahhh maybe not babe      4.0  1.649342e+09   
3                                                NaN    236.0  1.417630e+09   
4   \n\nLike... why? How is that funny? How does ...      6.0  1.569280e+09   

   num_comments  label  
0           4.0    0.0  
1           NaN    0.0  
2          12.0    0.0  
3          33.0    0.0  
4           4.0    0.0  


In [63]:
#check the data describe 
data.describe()

,upvotes,created_utc,num_comments,label
count,98826.000000,9.882600e+04,94297.000000,98826.000000
mean,62.315555,1.566569e+09,15.217069,0.194443
std,937.347581,6.977745e+07,71.638197,0.395773
min,4.000000,1.202084e+09,1.000000,0.000000
25%,5.000000,1.552254e+09,4.000000,0.000000
50%,7.000000,1.578218e+09,7.000000,0.000000
75%,11.000000,1.611584e+09,14.000000,0.000000
max,62899.000000,1.672531e+09,7880.000000,1.000000


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98826 entries, 0 to 98825
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         98826 non-null  object 
 1   body          80371 non-null  object 
 2   upvotes       98826 non-null  float64
 3   created_utc   98826 non-null  float64
 4   num_comments  94297 non-null  float64
 5   label         98826 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.5+ MB


### Data Cleaning

In [65]:
#Handle missing values
#check to see if there are null values in percentage form
data.isnull().sum() / len(data) * 100

title            0.000000
body            18.674236
upvotes          0.000000
created_utc      0.000000
num_comments     4.582802
label            0.000000
dtype: float64

Because the dataset is relatively large, I decides to drop rows that have missing values 

In [66]:
clean_data = data.dropna()


In [67]:
# Verify that there are no more missing values
print(data.isnull().sum())

title               0
body            18455
upvotes             0
created_utc         0
num_comments     4529
label               0
dtype: int64


Remove duplicates

In [68]:
# Remove duplicates and assign it back to the dataframe
clean_data = clean_data.drop_duplicates()



In [69]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77051 entries, 2 to 98825
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         77051 non-null  object 
 1   body          77051 non-null  object 
 2   upvotes       77051 non-null  float64
 3   created_utc   77051 non-null  float64
 4   num_comments  77051 non-null  float64
 5   label         77051 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.1+ MB


having 77,051 rows seemed to small so I opted to fill the missing rows in the body column with "'no content" to presere the data set instead.


In [71]:
# Make sure to initialize 'data_main' with 'data' first
data_main = data.copy()  # This creates a copy of the original DataFrame

# Now you can fill missing values in 'body' column for the new 'data_main'
data_main['body'] = data_main['body'].fillna("No content")

# Check the DataFrame info again to confirm the change
data_main.info() 




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98826 entries, 0 to 98825
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         98826 non-null  object 
 1   body          98826 non-null  object 
 2   upvotes       98826 non-null  float64
 3   created_utc   98826 non-null  float64
 4   num_comments  94297 non-null  float64
 5   label         98826 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.5+ MB


In [72]:
#check for null values 
# Verify that there are no more missing values
data_main.isnull().sum()

title              0
body               0
upvotes            0
created_utc        0
num_comments    4529
label              0
dtype: int64

In [81]:
# Now you can fill missing values in 'body' column for the new 'data_main'
data_main['num_comments'] = data_main['num_comments'].fillna(0)

# Check the DataFrame info again to confirm the change
data_main.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98826 entries, 0 to 98825
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         98826 non-null  object 
 1   body          98826 non-null  object 
 2   upvotes       98826 non-null  float64
 3   created_utc   98826 non-null  float64
 4   num_comments  98826 non-null  float64
 5   label         98826 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.5+ MB


In [82]:
#check for null values 
# Verify that there are no more missing values
data_main.isnull().sum()

title           0
body            0
upvotes         0
created_utc     0
num_comments    0
label           0
dtype: int64

In [84]:
# Convert 'created_utc' column from UTC epoch time to datetime
data_main['created_utc'] = pd.to_datetime(data_main['created_utc'], unit='s')

# To see the updated DataFrame
print(data_main.head(10))


                                               title  \
0                        Day 18 of doing 50 push-ups   
1  There isn’t a better feeling than finishing yo...   
2  LMAOOO I can only get this guy to talk to me i...   
3                       This isn't going to work out   
4  Am I the only guy who found it hurtful as a ch...   
5                                   Phew , close one   
6              Listen to my favorite song part 2 🤩🤩🤩   
7                                 Hard core cuddling   
8                      Don't communicate with women.   
9       Do you guys just ever want to be left alone?   

                                                body  upvotes  \
0                                         No content      4.0   
1  Then your teacher hits you with that “ Good jo...      7.0   
2                              Yeahhh maybe not babe      4.0   
3                                         No content    236.0   
4   \n\nLike... why? How is that funny? How does ...      